In [2]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import os
import io
import gradio as gr

# Authenticate client

prediction_key = "07051a00c54f429aacf110c380a4e3fd"
prediction_endpoint = "https://team1thetteamcv-prediction.cognitiveservices.azure.com/"
project_id = "240ecda2-da48-4e42-8e67-107e60f73f3d"
model_name = "Iteration2"

credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(endpoint=prediction_endpoint, credentials=credentials)

def process_image(image):
    # 이미지 리사이즈
    max_size = (1024, 1024)
    image.thumbnail(max_size)
    
    # JPEG로 변환
    byte_arr = io.BytesIO()
    image.save(byte_arr, format='JPEG')
    return byte_arr.getvalue()

def predict_image(input_img):
    processed_image = process_image(input_img)
    h, w = input_img.size
    
    results = predictor.classify_image(project_id, model_name, processed_image)  # 이미지 분류용 메서드
    
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.axis('off')
    
    draw = ImageDraw.Draw(input_img)
    lineWidth = int(w / 100)
    color = 'magenta'
    
    for prediction in results.predictions:
        if (prediction.probability * 100) > 50:
            draw.text((10, 10), f"{prediction.tag_name}: {prediction.probability * 100:.2f}%", fill=color)
            break
    
    ax.imshow(input_img)
    
    # 예측 결과 텍스트 생성
    predictions_text = "\n".join(
        f"{prediction.tag_name}: {prediction.probability * 100:.2f}%"
        for prediction in results.predictions
        if (prediction.probability * 100) > 50
    )
    
    return fig, predictions_text

result_output = gr.Textbox(label="Result")

demo = gr.Interface(
    fn=predict_image, 
    inputs=gr.Image(type='pil'), 
    outputs=[gr.Plot(), result_output]
)
demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7890

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
